In [78]:
from collections import Counter
import networkx as nx
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import itertools
import seaborn as sns
import ast
import numpy as np
import sklearn as skl
%matplotlib inline

In [79]:
df_recipe_raw = pd.read_csv("/Users/sagarmahajan/Desktop/Year4/FYP/archive/raw-data_recipe.csv")
df_recipe = pd.read_csv("/Users/sagarmahajan/Desktop/Year4/FYP/archive/core-data_recipe.csv")
df_test = pd.read_csv("/Users/sagarmahajan/Desktop/Year4/FYP/archive/core-data-test_rating.csv")
df_train = pd.read_csv("/Users/sagarmahajan/Desktop/Year4/FYP/archive/core-data-train_rating.csv")
df_rating = pd.read_csv("/Users/sagarmahajan/Desktop/Year4/FYP/archive/core-data-valid_rating.csv")
df_interactions = pd.read_csv("/Users/sagarmahajan/Desktop/Year4/FYP/archive/raw-data_interaction.csv")

In [43]:
print(df_interactions.groupby('recipe_id')['user_id'].nunique())

recipe_id
6663       44
6664       47
6665      172
6666      356
6667       87
         ... 
262646      1
262797      4
262815      1
262850      1
262956      2
Name: user_id, Length: 49698, dtype: int64


In [7]:
df_interactions.describe()

,user_id,recipe_id,rating
count,3.794003e+06,3.794003e+06,3.794003e+06
mean,4.860243e+06,6.112180e+04,4.513192e+00
std,5.010315e+06,7.178721e+04,8.754177e-01
min,1.600000e+01,6.663000e+03,1.000000e+00
25%,1.404674e+06,1.423500e+04,4.000000e+00
50%,2.954489e+06,2.339000e+04,5.000000e+00
75%,6.384689e+06,7.864900e+04,5.000000e+00
max,2.346374e+07,2.629560e+05,5.000000e+00


In [8]:
df_recipe_raw = df_recipe_raw.drop(columns = ['image_url','cooking_directions'])

In [9]:
df_recipe_raw['reviews'][0]

'{8542392: {\'rating\': 5, \'followersCount\': 11, \'madeRecipesCount\': 18, \'favoritesCount\': 200, \'dateLastModified\': u\'2017-04-22T12:46:43.663\', \'text\': u"Best breakfast ever! I ran out of paprika while seasoning, so I used garlic piercer on the other half of the batch. Very good! Can\'t wait to make it again... and figure out how to use the drippings!", \'followingCount\': 0}, 11174581: {\'rating\': 5, \'followersCount\': 8, \'madeRecipesCount\': 55, \'favoritesCount\': 101, \'dateLastModified\': u\'2013-06-20T15:50:25.96\', \'text\': u"Awesome!\\nIt\'s amazing.", \'followingCount\': 0}, 8262477: {\'rating\': 5, \'followersCount\': 0, \'madeRecipesCount\': 1, \'favoritesCount\': 52, \'dateLastModified\': u\'2015-02-14T07:27:51.307\', \'text\': u\'The flavors came together well and it really was simple to prepare. My husband and I both enjoyed it!\', \'followingCount\': 0}}\n'

In [10]:
df_recipe_raw.shape

(49698, 7)

In [44]:
df_recipe_raw.isnull().sum()

recipe_id             0
recipe_name           0
aver_rate             0
image_url             0
review_nums           0
ingredients           0
cooking_directions    0
nutritions            0
reviews               0
dtype: int64

In [45]:
def avg_rate(col):
    return f'{col:.2f}'
df_recipe_raw.aver_rate = df_recipe_raw.aver_rate.apply(avg_rate)
df_recipe_raw.head()

,recipe_id,recipe_name,aver_rate,image_url,review_nums,ingredients,cooking_directions,nutritions,reviews
0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.76,https://images.media-allrecipes.com/userphotos...,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'...","{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,Chicken Pesto Paninis,4.62,https://images.media-allrecipes.com/userphotos...,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,Potato Bacon Pizza,4.50,https://images.media-allrecipes.com/userphotos...,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2945555: {'rating': 5, 'followersCount': 6690..."


### Cleaning Nutrition Data

In [14]:
df_recipe_raw['nutritions'][0]

"{u'niacin': {u'hasCompleteData': False, u'name': u'Niacin Equivalents', u'amount': 9.319291, u'percentDailyValue': u'72', u'displayValue': u'9', u'unit': u'mg'}, u'sugars': {u'hasCompleteData': True, u'name': u'Sugars', u'amount': 0.09355932, u'percentDailyValue': u'0', u'displayValue': u'0.1', u'unit': u'g'}, u'sodium': {u'hasCompleteData': True, u'name': u'Sodium', u'amount': 2017.13, u'percentDailyValue': u'81', u'displayValue': u'2017', u'unit': u'mg'}, u'carbohydrates': {u'hasCompleteData': True, u'name': u'Carbohydrates', u'amount': 1.797819, u'percentDailyValue': u'< 1', u'displayValue': u'1.8', u'unit': u'g'}, u'vitaminB6': {u'hasCompleteData': False, u'name': u'Vitamin B6', u'amount': 0.2329798, u'percentDailyValue': u'15', u'displayValue': u'< 1', u'unit': u'mg'}, u'calories': {u'hasCompleteData': True, u'name': u'Calories', u'amount': 308.1481, u'percentDailyValue': u'15', u'displayValue': u'308', u'unit': u'kcal'}, u'thiamin': {u'hasCompleteData': False, u'name': u'Thiamin

In [80]:
list_of_dict = []

for row in df_recipe_raw.nutritions:
    list_of_dict.append(ast.literal_eval(row))
list_of_dict[1]

{'niacin': {'hasCompleteData': False,
  'name': 'Niacin Equivalents',
  'amount': 15.6016,
  'percentDailyValue': '120',
  'displayValue': '16',
  'unit': 'mg'},
 'sugars': {'hasCompleteData': False,
  'name': 'Sugars',
  'amount': 19.84146,
  'percentDailyValue': '0',
  'displayValue': '19.8',
  'unit': 'g'},
 'sodium': {'hasCompleteData': False,
  'name': 'Sodium',
  'amount': 2606.764,
  'percentDailyValue': '104',
  'displayValue': '2607',
  'unit': 'mg'},
 'carbohydrates': {'hasCompleteData': True,
  'name': 'Carbohydrates',
  'amount': 32.08176,
  'percentDailyValue': '10',
  'displayValue': '32.1',
  'unit': 'g'},
 'vitaminB6': {'hasCompleteData': False,
  'name': 'Vitamin B6',
  'amount': 1.328631,
  'percentDailyValue': '83',
  'displayValue': '1',
  'unit': 'mg'},
 'calories': {'hasCompleteData': True,
  'name': 'Calories',
  'amount': 371.7219,
  'percentDailyValue': '19',
  'displayValue': '372',
  'unit': 'kcal'},
 'thiamin': {'hasCompleteData': False,
  'name': 'Thiamin',

In [76]:
calories_list = []
fat_list = []
saturatedFat_list = []
sodium_list = []
sugars_list = []

for x in range(len(list_of_dict)):
    calories_list.append(list_of_dict[x]['calories']['displayValue'])
    fat_list.append(list_of_dict[x]['fat']['displayValue'])
    saturatedFat_list.append(list_of_dict[x]['saturatedFat']['displayValue'])
    sodium_list.append(list_of_dict[x]['sodium']['displayValue'])
    sugars_list.append(list_of_dict[x]['sugars']['displayValue'])
y_true = np.array([0, 0, 1, 1]) 
y_score = np.array([0.1, 0.4, 0.35, 0.8])

In [81]:
data = {'calories': calories_list, 'fat': fat_list, 'saturatedFat': saturatedFat_list, 
       'sodium': sodium_list, 'sugars': sugars_list}

df = pd.DataFrame(data)
df.index = df_recipe_raw['recipe_id']
df.head()

,calories,fat,saturatedFat,sodium,sugars
recipe_id,,,,,
222388,308,23.6,7.7,2017,0.1
240488,372,11.7,3.6,2607,19.8
218939,335,23.6,5.7,763,0.2
87211,641,29.4,10.9,1076,2
245714,163,7.6,4,190,2.5


In [82]:
df_recipe_raw = df_recipe_raw.set_index("recipe_id")


In [83]:
df_recipe_raw['ingredients'] = df_recipe_raw['ingredients'].apply(lambda x: x.split("^"))


In [16]:
print(df_recipe_raw.loc[df_recipe_raw['recipe_id'] == 170724 , 'ingredients'].values)

[list(['butternut squash', 'carrots', 'large sweet potato', 'rutabaga peeled and cut into 1-inch pieces', 'parsnips peeled and cubed', 'turnips peeled and cut into 1-inch dice', 'extra virgin olive oil', 'kosher salt and pepper to taste'])]


In [84]:
df_30000 = df_train.head(30000).copy()

In [85]:
df_30000

,user_id,recipe_id,rating,dateLastModified
0,5215572,17991,5,2010-08-25T14:38:53.84\n
1,5215572,170724,4,2010-09-09T14:04:45.733\n
2,5215572,18045,5,2010-08-16T14:51:25.833\n
3,3622615,60598,4,2009-03-15T12:10:20.85\n
4,1313770,47519,5,2005-10-04T15:43:36.653\n
...,...,...,...,...
29995,2498083,27072,4,2009-02-28T16:16:22.723\n
29996,2498083,22338,4,2007-09-24T08:40:17.013\n
29997,2498083,143090,5,2010-02-06T15:01:58.93\n
29998,2498083,33352,4,2008-03-09T17:57:47.303\n


In [87]:
df_30000['calories'] = df.loc[df_30000['recipe_id'], 'calories'].values
df_30000['fat'] = df.loc[df_30000['recipe_id'], 'fat'].values
df_30000['saturatedFat'] = df.loc[df_30000['recipe_id'], 'saturatedFat'].values
df_30000['sodium'] = df.loc[df_30000['recipe_id'], 'sodium'].values
df_30000['sugars'] = df.loc[df_30000['recipe_id'], 'sugars'].values

In [88]:
df_30000['ingredients'] = df_recipe_raw.loc[df_30000['recipe_id'], 'ingredients'].values
df_30000['recipe_name'] = df_recipe_raw.loc[df_30000['recipe_id'], 'recipe_name'].values

In [89]:
df_30000 = df_30000.drop(columns = ['dateLastModified'])

In [29]:
df_recipe_raw.head(50)

,recipe_id,recipe_name,aver_rate,image_url,review_nums,ingredients,cooking_directions,nutritions,reviews
0,222388,Homemade Bacon,5.000000,https://images.media-allrecipes.com/userphotos...,3,"[pork belly, smoked paprika, kosher salt, grou...",{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,https://images.media-allrecipes.com/userphotos...,29,"[sauerkraut drained, Granny Smith apples slice...",{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,https://images.media-allrecipes.com/userphotos...,12,"[chicken wings, sprigs rosemary, head garlic, ...","{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'...","{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,Chicken Pesto Paninis,4.625000,https://images.media-allrecipes.com/userphotos...,163,"[focaccia bread quartered, prepared basil pest...",{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,Potato Bacon Pizza,4.500000,https://images.media-allrecipes.com/userphotos...,2,"[red potatoes, strips bacon, Sauce:, heavy whi...",{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2945555: {'rating': 5, 'followersCount': 6690..."
5,218545,Latin-Inspired Spicy Cream Chicken Stew,4.605769,https://images.media-allrecipes.com/userphotos...,85,"[skinless boneless chicken breast halves, dice...",{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8044800: {'rating': 5, 'followersCount': 0, '..."
6,20453,Reuben Sandwich I,4.250000,https://images.media-allrecipes.com/userphotos...,15,"[rye bread, butter, thinly sliced corned beef,...",{'directions': u'Cook\n5 m\nReady In\n5 m\nHea...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2650112: {'rating': 5, 'followersCount': 7, '..."
7,244856,Turkey Black Bean Burgers,4.857143,https://images.media-allrecipes.com/userphotos...,7,"[extra lean ground turkey, cooked black beans ...",{'directions': u'Prep\n10 m\nCook\n8 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{17766085: {'rating': 4, 'followersCount': 0, ..."
8,22402,Cranberry Pork Chops II,4.427481,https://images.media-allrecipes.com/userphotos...,100,"[pork chops, fresh, white sugar, salt, ground ...",{'directions': u'Prep\n25 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{3303939: {'rating': 5, 'followersCount': 0, '..."
9,258163,Schnitzel Sandwich,5.000000,https://images.media-allrecipes.com/userphotos...,1,"[skinless boneless chicken breasts, salt and g...",{'directions': u'Prep\n20 m\nCook\n20 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{3567821: {'rating': 5, 'followersCount': 9199..."


In [90]:
df_30000.head(50)

,user_id,recipe_id,rating,calories,fat,saturatedFat,sodium,sugars,ingredients,recipe_name
0,5215572,17991,5,463,24.3,12.5,971,12.2,"[green bell peppers, salt to taste, ground bee...",Stuffed Green Peppers I
1,5215572,170724,4,210,6,0.9,121,11.2,"[butternut squash, carrots, large sweet potato...",Sean's Mommy's Roasted Root Vegetables
2,5215572,18045,5,196,14.8,7.7,463,1.7,"[sliced yellow squash, chopped onion, buttery ...",Yellow Squash Casserole
3,3622615,60598,4,462,31.1,15.3,1434,9.3,"[vegetable oil, small onion, minced fresh ging...",Vegetarian Korma
4,1313770,47519,5,468,18.7,7.7,678,36.5,"[butter, fresh mushrooms sliced, cream cheese,...",Cream Cheese Chicken
5,1313770,42964,5,376,21.9,7.1,1249,15.5,[thinly sliced Scotch fillet (chuck eye steaks...,Awesome Korean Steak
6,1313770,47822,5,241,13.1,5.6,339,13.6,"[brown sugar, ground ginger, dried tarragon, g...",Grilled Lamb with Brown Sugar Glaze
7,1313770,11815,5,435,18.4,6.2,544,1,"[uncooked penne pasta, extra virgin olive oil,...",Penne with Spicy Vodka Tomato Cream Sauce
8,1313770,12780,5,590,23.7,13.5,317,3.7,"[linguine pasta, butter, diced onion, minced g...",Shrimp Linguine Alfredo
9,1313770,150711,5,187,13.7,4.4,266,0.3,"[peanut oil, butter, unpopped popcorn, salt]",Movie Star Popcorn


In [54]:
df_10000.to_csv('core_data_train_full.csv', encoding='utf-8')

In [91]:
df_30000.to_csv('recipe_data_full.csv', encoding='utf-8')